# CNN for Regression from Noiseless Images to Labels

## Stage 3: Retraining the Best Model on the Full Dataset (200k Samples)

In [1]:
import json
from pprint import pprint
import numpy as np
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
import tensorflow_addons as tfa

%run xception.ipynb

In [2]:
dataset = "../../data/data_v1.npz"

with np.load(dataset) as data:
    print("Available variables:", data.files)
    image_nonoise = data["img_nonoise"]
    label = data["label"]

Available variables: ['img', 'img_nonoise', 'label', 'psf_r', 'snr', 'sigma']


In [3]:
# Use unscaled noiseless images. Let TF normalize the data.
X = image_nonoise[..., np.newaxis]

# Scale the labels between -1 and 1 using the full dataset. We know the ranges of generated labels.
scaler = MinMaxScaler(feature_range=(-1, 1))
y = scaler.fit_transform(label)

In [4]:
# Show the parameters of the best model
with open("noiseless_params.json") as f:
    pprint(json.load(f))

{'image_block_1/xception_block_1/activation': 'selu',
 'image_block_1/xception_block_1/conv2d_num_filters': 64,
 'image_block_1/xception_block_1/initial_strides': 2,
 'image_block_1/xception_block_1/kernel_size': 5,
 'image_block_1/xception_block_1/num_residual_blocks': 7,
 'image_block_1/xception_block_1/pooling': 'avg',
 'image_block_1/xception_block_1/sep_num_filters': 256,
 'optimizer': 'adam',
 'regression_head_1/dropout_rate': 0}


In [5]:
# Load the best model. SeparableConv2D layers form Xception blocks.
model = tf.keras.models.load_model('noiseless_20k.tf', custom_objects=ak.CUSTOM_OBJECTS)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
normalization (Normalization)   (None, 64, 64, 1)    3           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 64)   1664        normalization[0][0]              
__________________________________________________________________________________________________
separable_conv2d (SeparableConv (None, 32, 32, 256)  17216       conv2d[0][0]                     
______________________________________________________________________________________________

In [6]:
# Adam with the default settings was chosen by Autokeras as the optimizer
model.optimizer.get_config()

{'name': 'Adam',
 'learning_rate': 0.001,
 'decay': 0.0,
 'beta_1': 0.9,
 'beta_2': 0.999,
 'epsilon': 1e-07,
 'amsgrad': False}

In [7]:
# Setup callbacks. TQDM is used due to issues with the default progress bar on my TF2.1 installation.
early_stopping = tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
tqdm_callback = tfa.callbacks.TQDMProgressBar()

# Optional: reinitialize the weights
# ...
# Optional: reset the optimizer state
model.compile(optimizer="adam", loss="mse", metrics=["mse"])

# Train the best model on the full dataset
history = model.fit(X, y, epochs=100, validation_split=0.1, verbose=0, callbacks=[early_stopping, tqdm_callback])

Epoch 1/100



Epoch 2/100



Epoch 3/100



Epoch 4/100



Epoch 5/100



Epoch 6/100



Epoch 7/100



Epoch 8/100



Epoch 9/100


In [9]:
# Save the fully trained model
tf.keras.models.save_model(model, 'noiseless_200k.tf', save_format='tf')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: noiseless_full.tf/assets


In [4]:
# Optionally: reload the saved model
model = tf.keras.models.load_model('noiseless_200k.tf')

In [5]:
# Evaluate the model on the validation set
n_train = int(label.shape[0] * 0.9)
X_val = X[n_train:]
label_val = label[n_train:]

# Predict the labels and convert them to the original scale
predictions = model.predict(X_val)
predictions = scaler.inverse_transform(predictions)

# Compute RMSE for each label individually
np.set_printoptions(suppress=True)
rmse = np.sqrt(((label_val - predictions)**2).mean(axis=0))
rmse

array([489.7916    ,   0.03671913,   0.00371554,   0.00392448,
         0.00340343], dtype=float32)

In [6]:
# Show the standard deviations of the labels for comparison
label_val.std(axis=0)

array([48742.957     ,     1.5922561 ,     0.14061895,     0.2791248 ,
           0.27738345], dtype=float32)

In [7]:
# RMSE / std. dev
rmse / label_val.std(axis=0)

array([0.01004846, 0.02306107, 0.02642274, 0.01405993, 0.01226977],
      dtype=float32)